In [21]:
import os
import requests
import pandas as pd
import json
import asyncio

from datetime import datetime
from datetime import timedelta
import sys
sys.path.append("../../")
from app.utility.helps import Bob
from app.utility.fab2 import File_Table_Management

In [22]:
os.getcwd()

'/home/brandon/projects/fabric/monitor/FabricMonitor/app/notebooks'

In [12]:
bob = Bob()
# get POWER BI context and settings -- this call must be synchronous
settings = bob.get_settings()
headers = bob.get_context()

FF = File_Table_Management(
    tenant_id=settings['ServicePrincipal']['TenantId'],
    client_id=settings['ServicePrincipal']['AppId'],
    client_secret=settings['ServicePrincipal']['AppSecret'],
    workspace_name=settings['WorkspaceName']
)

lakehouse_catalog = f"{settings['LakehouseName']}.Lakehouse/Files/catalog/"

##################### INTIALIZE THE CONFIGURATION #####################
today = datetime.now()


In [13]:

snapshots =f"snapshots/{today.strftime('%Y')}/{today.strftime('%m')}/{today.strftime('%d')}"
## check if file already exists
#lakehouse_dir = f"{lakehouse_catalog}{snapshots}"
lakehouse_dir = f"{lakehouse_catalog}snapshots/2024/02/15"

print(f"Checking if {lakehouse_dir} exists")

fsc = FF.get_file_system_client(
    tenant_id=settings['ServicePrincipal']['TenantId'],
    client_id=settings['ServicePrincipal']['AppId'],
    client_secret=settings['ServicePrincipal']['AppSecret'],
    workspace_name=settings['WorkspaceName']   
)


Checking if FabricLake.Lakehouse/Files/catalog/snapshots/2024/02/15 exists


In [14]:

try:
    paths = fsc.get_paths(lakehouse_dir)
    for path in paths:
        print(f"Path: {path.name}")
        if "app.json" in path.name:
            print(f"File {path.name} already exists")
            exit(0)

except Exception as e:
    print(f"Error: {e} - continue with executing code")    


Path: FabricLake.Lakehouse/Files/catalog/snapshots/2024/02/15/apps.json


In [ ]:

dc = await FF.create_directory(file_system_client=FF.fsc, directory_name=lakehouse_dir)
